In [1]:
from llvm_ml.torch import BasicBlockDataset
banned_ids = []
dataset = BasicBlockDataset("./data/ryzen3600_v16.cbuf", masked=False, banned_ids=banned_ids, prefilter=True)
print(f"Training with {len(dataset)} samples")

Training with 347988 samples


In [12]:
from model.GraphBERT import ThroughputEstimator
from torch_geometric.loader import DataLoader
import torch
from lightning.pytorch.loggers import TensorBoardLogger
import lightning.pytorch as pl
from model import GraphBERT
from lightning.pytorch.callbacks import ModelSummary, LearningRateMonitor

import importlib
importlib.reload(GraphBERT)

import warnings
warnings.filterwarnings('ignore', category=UserWarning, message='TypedStorage is deprecated')

batch_size = 256
hidden_dim = 128
emb_size = 64

num_training = int(0.7 * len(dataset))
num_val = len(dataset) - num_training

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [num_training, num_val])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=6, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=6, drop_last=True)

config = {
    'num_opcodes': dataset.num_opcodes,
    'embedding_size': emb_size,
    'batch_size': batch_size,
    'hidden_dim': hidden_dim,
    'learning_rate': 3e-4,
    'mode': 'pretrain',
    #'learning_rate': 2e-5,
    #'mode': 'regression',
    'num_encoders': 6,
    'num_head_encoder': 8
}

model = ThroughputEstimator(config)
# model.load_state_dict(torch.load("data/pretrained_mcmlm.ckpt")['state_dict'])

logger = TensorBoardLogger("runs", name="bert")
logger.log_graph(model)
callbacks = [
    ModelSummary(max_depth=-1),
    LearningRateMonitor(),
]
trainer = pl.Trainer(max_epochs=25,
                     logger=logger,
                     precision='16-mixed',
                     callbacks=callbacks,
                     # fast_dev_run=True,
                     #overfit_batches=1,
                     #log_every_n_steps=1,
                     )
trainer.fit(model, train_loader, val_loader)

Using 16bit Automatic Mixed Precision (AMP)
Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                             | Type               | Params
-------------------------------------------------------------------------
0  | embedding                        | MCEmbedding        | 1.3 M 
1  | embed

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=25` reached.


In [8]:
torch.save({'state_dict': model.state_dict()}, "data/pretrained_mcmlm.ckpt")

In [ ]:
# torch.save({'state_dict': model.state_dict()}, "data/ryzen3600.ckpt")